# Write Audit Publish

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Create a Table with initial data

In [ ]:
create or replace schema wap;

create or replace table wap_data as 
select 1 as col_a;

## Validate the table

In [ ]:
select count(1) = 0 from wap_data where col_a is null;

## Publish the data as a view

In [ ]:
def publish_correct_data():
    session.use_schema("WAP")
    validation_time = session.sql("select current_timestamp()").first()[0]
    validation_passes = (session.sql(f"select count(1) = 0 from wap_data at (timestamp => to_timestamp_tz('{validation_time}')) where col_a is null")
                         .first()[0])
    if validation_passes:
        session.sql(f"""
        create or replace view wap_published as select * from wap_data at (timestamp => to_timestamp_tz('{validation_time}'))
        """).first()

In [ ]:
publish_correct_data()

In [ ]:
select * from wap_data

In [ ]:
select * from wap_published

## Insert some bad data

In [ ]:
insert into wap_data select to_number(null) as col_a

## Attempt publishing the view after the table modification

In [ ]:
publish_correct_data()

In [ ]:
select * from wap_published

In [ ]:
select * from wap_data